#### Rock Shi Churn Prediction Capstone

In [1]:
# Importing all packages needed

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import when
from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import *
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StandardScaler
from sklearn.metrics import confusion_matrix
from pyspark.sql.types import *

In [2]:
# Reading in train and test datasets from hadoop
churn_train = spark.read.csv('/spark/input/churn-bigml-80.csv', inferSchema = True, header = True)
churn_test = spark.read.csv('/spark/input/churn-bigml-20.csv', inferSchema = True, header = True)

In [3]:
# Displaying first five rows of train dataset
churn_train.show(5)

+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+
|State|Account length|Area code|International plan|Voice mail plan|Number vmail messages|Total day minutes|Total day calls|Total day charge|Total eve minutes|Total eve calls|Total eve charge|Total night minutes|Total night calls|Total night charge|Total intl minutes|Total intl calls|Total intl charge|Customer service calls|Churn|
+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+
|   